In [43]:
from tensorflow import keras
import tensorflow as tf
import os,datetime
import tensorflow_datasets as tfds
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [60]:


class Resnet50:

    def __init__(self, input_shape = (64, 64, 3), classes = 10):
        """
        Implementation of the ResNet50 architecture based on paper https://arxiv.org/abs/1512.03385:

        Arguments:
        input_shape -- shape of the images of the dataset
        classes -- integer, number of classes

        Returns:
        model -- a Model() instance in Keras
        """
        

        X_input = Input(shape=input_shape)

        X = ZeroPadding2D((3, 3))(X_input)

        X = Conv2D(64, (7, 7), strides = (2, 2))(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        X = MaxPooling2D((3, 3), strides=(2, 2))(X)

        X = self.convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
        X = self.identity_block(X, 3, [64, 64, 256])
        X = self.identity_block(X, 3, [64, 64, 256])

        X = self.convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
        X = self.identity_block(X, 3, [128, 128, 512])
        X = self.identity_block(X, 3, [128, 128, 512])
        X = self.identity_block(X, 3, [128, 128, 512])

        X = self.convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
        X = self.identity_block(X, 3, [256, 256, 1024])
        X = self.identity_block(X, 3, [256, 256, 1024])
        X = self.identity_block(X, 3, [256, 256, 1024])
        X = self.identity_block(X, 3, [256, 256, 1024])
        X = self.identity_block(X, 3, [256, 256, 1024])

        X = self.convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
        X = self.identity_block(X, 3, [512, 512, 2048])
        X = self.identity_block(X, 3, [512, 512, 2048])

        X = AveragePooling2D()(X)
        
        X = Flatten()(X)
        if classes > 2:
          X = Dense(classes, activation='softmax')(X)
        else:
          X = Dense(1, activation='sigmoid')(X)
        model = Model(inputs = X_input, outputs = X, name='ResNet50')
        self.model = model
        

    def compile(self, **kwargs):
        self.model.compile(**kwargs)


    def identity_block(self, X, f, filters):
        """
        Implementation of the identity block as defined in Figure 3
        
        Arguments:
        X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
        f -- integer, specifying the shape of the middle CONV's window for the main path
        filters -- python list of integers, defining the number of filters in the CONV layers of the main path
        stage -- integer, used to name the layers, depending on their position in the network
        block -- string/character, used to name the layers, depending on their position in the network
        
        Returns:
        X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
        """
        
        F1, F2, F3 = filters
        
        X_shortcut = X
        
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        
        X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)

        X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
        X = BatchNormalization(axis = 3)(X)

        X = Add()([X_shortcut, X]) 
        X = Activation('relu')(X)
        
        return X


    def convolutional_block(self, X, f, filters, s = 2):
        """
        Implementation of the convolutional block as defined in Figure 4
        
        Arguments:
        X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
        f -- integer, specifying the shape of the middle CONV's window for the main path
        filters -- python list of integers, defining the number of filters in the CONV layers of the main path
        stage -- integer, used to name the layers, depending on their position in the network
        block -- string/character, used to name the layers, depending on their position in the network
        s -- Integer, specifying the stride to be used
        
        Returns:
        X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
        """

        F1, F2, F3 = filters
        
        X_shortcut = X

        X = Conv2D(F1, (1, 1), strides = (s,s), padding = 'valid')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        
        X = Conv2D(F2, (f, f), strides = (1,1), padding = 'same')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)

        X = Conv2D(F3, (1, 1), strides = (1,1), padding = 'valid')(X)
        X = BatchNormalization(axis = 3)(X)
        
        X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), padding = 'valid')(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

        X = Add()([X, X_shortcut])
        X = Activation('relu')(X)
        
        return X



In [37]:
!rm -rf "/root/tensorflow_datasets"

In [38]:
#Loading the data from tensorflow_datasets
df, info = tfds.load('patch_camelyon', with_info = True, as_supervised = True)

#Getting the train, validation and test data
train_data = df['train']
valid_data = df['validation']
test_data = df['test']

Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incompleteRR6HJ0/patch_camelyon-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incompleteRR6HJ0/patch_camelyon-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incompleteRR6HJ0/patch_camelyon-validation.tfrecord


Dataset patch_camelyon downloaded and prepared to /root/tensorflow_datasets/patch_camelyon/2.0.0. Subsequent calls will reuse this data.


In [39]:
#A function to help scale the images
def preprocess(image, labels):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, labels

#Applying the preprocess function we the use of map() method
train_data = train_data.map(preprocess)
valid_data = valid_data.map(preprocess)
test_data = test_data.map(preprocess)

#Shuffling the train_data
buffer_size = 1000
train_data = train_data.shuffle(buffer_size)

#Batching and prefetching
batch_size = 128
train_data = train_data.batch(batch_size).prefetch(1)
valid_data = valid_data.batch(batch_size).prefetch(1)
test_data = test_data.batch(batch_size).prefetch(1)


In [40]:
#Seperating image and label into different variables
train_images, train_labels = next(iter(train_data))
valid_images, valid_labels = next(iter(valid_data))
test_images, test_labels  = next(iter(test_data))

#Checking the label shape
valid_labels.shape

#Checking the image shape
train_images.shape

TensorShape([128, 96, 96, 3])

In [61]:
model = Resnet50(input_shape = (96, 96, 3), classes = 2)

In [62]:
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['acc'])

In [63]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [64]:
history = model.model.fit( train_images, train_labels, epochs = 100, callbacks=[early_stopping_cb], validation_data = (valid_images, valid_labels), verbose=2)


Epoch 1/100


4/4 - 1s - loss: 2.8169 - acc: 0.5469 - val_loss: 0.6762 - val_acc: 0.5859
Epoch 2/100
4/4 - 0s - loss: 1.1103 - acc: 0.5938 - val_loss: 0.6904 - val_acc: 0.6406
Epoch 3/100
4/4 - 0s - loss: 0.5878 - acc: 0.6562 - val_loss: 0.7133 - val_acc: 0.4141
Epoch 4/100
4/4 - 0s - loss: 0.4520 - acc: 0.7812 - val_loss: 0.7166 - val_acc: 0.4141
Epoch 5/100
4/4 - 0s - loss: 0.4544 - acc: 0.7656 - val_loss: 0.7633 - val_acc: 0.4141
Epoch 6/100
4/4 - 0s - loss: 0.2200 - acc: 0.9062 - val_loss: 0.7946 - val_acc: 0.4141
Epoch 7/100
4/4 - 0s - loss: 0.1395 - acc: 0.9609 - val_loss: 0.8758 - val_acc: 0.4141
Epoch 8/100
4/4 - 0s - loss: 0.0598 - acc: 0.9766 - val_loss: 0.9447 - val_acc: 0.4141
Epoch 9/100
4/4 - 0s - loss: 0.0112 - acc: 0.9922 - val_loss: 1.0823 - val_acc: 0.4141
Epoch 10/100
4/4 - 0s - loss: 0.0054 - acc: 1.0000 - val_loss: 1.2520 - val_acc: 0.4141
Epoch 11/100
4/4 - 0s - loss: 0.0105 - acc: 0.9922 - val_loss: 1.4031 - val_acc: 0.4141
